# NTP Hybrids Simulation Visualization

This notebook is aligned to the paper _Novel Hybrid Nuclear Thermal Propulsion Concepts for Enabling Affordable, Routine Human Mars Missions_. It uses the same simplified 0-D/1-D modeling scope described in the paper's software section and surfaces the paper-specific mechanism ranges directly:

- `EANTR`: 10-30% ionization, 10-50 kV electrostatic grids, cruise-only augmentation.
- `ARENTP`: 10-100 kHz acoustic drivers, 2-5x heat-transfer enhancement.
- `SF-NTR`: 300-500 bar chamber pressure, full-path supercritical flow, optional `H2/CH4` blends.

The notebook builds the Rust crate, runs the requested hybrid case, and saves plots plus CSV outputs into the dated `output-ntp-hybrids-sim` folder.

In [ ]:
from pathlib import Path
import os
import shutil
import subprocess
import sys

def find_project_dir(search_root=Path('/content')):
    cargo_tomls = sorted(search_root.rglob('Cargo.toml'))
    matches = [
        path.parent
        for path in cargo_tomls
        if path.parent.name == 'ntp-hybrids-sim' and path.parent.parent.name == 'crates'
    ]
    if not matches:
        raise FileNotFoundError('Clone or upload the repository into /content before running this notebook.')
    return matches[0]

cargo_path = shutil.which('cargo')
if cargo_path is None:
    subprocess.run(['bash', '-lc', 'curl https://sh.rustup.rs -sSf | sh -s -- -y'], check=True)
    cargo_bin = Path.home() / '.cargo' / 'bin'
    os.environ['PATH'] = f"{cargo_bin}:{os.environ['PATH']}"

PROJECT_DIR = find_project_dir()
REPO_ROOT = PROJECT_DIR.parent.parent
OUTPUT_ROOT = REPO_ROOT / 'output-ntp-hybrids-sim'
OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

subprocess.run(['cargo', '--version'], check=True)
subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'pandas', 'matplotlib', 'ipywidgets'], check=True)
subprocess.run(['cargo', 'build'], cwd=PROJECT_DIR, check=True)

print(f'Project directory: {PROJECT_DIR}')
print(f'Output root: {OUTPUT_ROOT}')


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

try:
    from google.colab import output as colab_output
    colab_output.enable_custom_widget_manager()
except Exception:
    pass

plt.style.use('seaborn-v0_8-whitegrid')

PAPER_REFERENCE = pd.DataFrame([
    {
        'hybrid': 'EANTR',
        'mechanism': 'Partial ionization + downstream electrostatic acceleration',
        'paper_isp_range_s': '1100-2000+',
        'paper_pmf_range': '40-55%',
        'paper_transit_range_days': '80-120',
        'key_parameters': 'Ionization 10-30%, grids 10-50 kV, electrostatic cruise mode',
    },
    {
        'hybrid': 'ARENTP',
        'mechanism': 'Acoustic resonance for boundary-layer disruption and heat-transfer gain',
        'paper_isp_range_s': '1080-1260',
        'paper_pmf_range': '48-55%',
        'paper_transit_range_days': '100-140',
        'key_parameters': 'Acoustic drivers 10-100 kHz, heat-transfer enhancement 2-5x',
    },
    {
        'hybrid': 'SF-NTR',
        'mechanism': 'Full-path supercritical propellant with high chamber pressure',
        'paper_isp_range_s': '1050-1200',
        'paper_pmf_range': '50-55%',
        'paper_transit_range_days': '100-150',
        'key_parameters': 'Chamber pressure 300-500 bar, supercritical heat transfer 3-5x',
    },
])

def latest_output_dir():
    dated_dirs = sorted(path for path in OUTPUT_ROOT.iterdir() if path.is_dir())
    if not dated_dirs:
        raise FileNotFoundError('No dated output folders found. Run the simulation first.')
    return dated_dirs[-1]

def hybrid_key(hybrid):
    return hybrid.lower().replace('-', '_')

def run_simulation(hybrid, dv=5500.0, runs=360):
    subprocess.run(
        ['cargo', 'run', '--', '--hybrid', hybrid, '--dv', f'{dv:.1f}', '--runs', str(runs)],
        cwd=PROJECT_DIR,
        check=True,
    )
    return latest_output_dir()

def load_results(output_dir, hybrid):
    key = hybrid_key(hybrid)
    summary = pd.read_csv(output_dir / 'summary.csv')
    summary = summary.loc[summary['hybrid'] == hybrid].reset_index(drop=True)
    return {
        'summary': summary,
        'paper': PAPER_REFERENCE.loc[PAPER_REFERENCE['hybrid'] == hybrid].reset_index(drop=True),
        'sweep': pd.read_csv(output_dir / f'isp_sweep_{key}.csv'),
        'mechanism': pd.read_csv(output_dir / f'mechanism_sweep_{key}.csv'),
        'monte': pd.read_csv(output_dir / f'monte_carlo_{key}.csv'),
        'profile': pd.read_csv(output_dir / f'thrust_profile_{key}.csv'),
    }

def save_plot(fig, path):
    fig.tight_layout()
    fig.savefig(path, dpi=200, bbox_inches='tight')


In [ ]:
hybrid_dropdown = widgets.Dropdown(
    options=['EANTR', 'ARENTP', 'SF-NTR'],
    value='EANTR',
    description='Hybrid',
)
dv_slider = widgets.FloatSlider(
    value=5500.0,
    min=4500.0,
    max=6000.0,
    step=100.0,
    description='Delta-v',
    readout_format='.0f',
    continuous_update=False,
)
run_button = widgets.Button(description='Run simulation', button_style='info')
output = widgets.Output()

def render_plots(output_dir, hybrid, frames):
    key = hybrid_key(hybrid)
    summary = frames['summary']
    paper = frames['paper']
    sweep = frames['sweep']
    mechanism = frames['mechanism']
    monte = frames['monte']
    profile = frames['profile']

    fig1, ax1 = plt.subplots(figsize=(8, 5))
    ax1.plot(sweep['isp_s'], sweep['rocket_propellant_fraction'], color='tab:blue', linewidth=2)
    ax1.axhline(
        summary.loc[0, 'baseline_reference_propellant_fraction'],
        color='tab:red',
        linestyle='--',
        label='Baseline 65% reference',
    )
    ax1.axhspan(
        summary.loc[0, 'global_hybrid_reference_propellant_fraction_min'],
        summary.loc[0, 'global_hybrid_reference_propellant_fraction_max'],
        color='tab:green',
        alpha=0.15,
        label='Global hybrid band 35-45%',
    )
    ax1.axhspan(
        summary.loc[0, 'paper_propellant_fraction_min'],
        summary.loc[0, 'paper_propellant_fraction_max'],
        color='tab:orange',
        alpha=0.18,
        label='Paper concept band',
    )
    ax1.set_title(f'{hybrid} mass fraction vs Isp')
    ax1.set_xlabel('Isp [s]')
    ax1.set_ylabel('Propellant mass fraction')
    ax1.legend()
    save_plot(fig1, output_dir / f'mass_fraction_vs_isp_{key}.png')
    plt.show()
    plt.close(fig1)

    fig2, ax2 = plt.subplots(figsize=(8, 5))
    ax2.plot(mechanism['driver_value'], mechanism['effective_isp_s'], color='tab:cyan', marker='o', linewidth=2)
    ax2.axhspan(
        summary.loc[0, 'paper_isp_min_s'],
        summary.loc[0, 'paper_isp_max_s'],
        color='tab:blue',
        alpha=0.12,
        label='Paper Isp band',
    )
    ax2.set_title(f'{hybrid} mechanism driver vs effective Isp')
    ax2.set_xlabel(f"{mechanism.loc[0, 'driver_name']} [{mechanism.loc[0, 'driver_units']}]")
    ax2.set_ylabel('Effective Isp [s]')
    ax2.legend()
    save_plot(fig2, output_dir / f'mechanism_driver_vs_isp_{key}.png')
    plt.show()
    plt.close(fig2)

    fig3, ax3 = plt.subplots(figsize=(8, 5))
    ax3.hist(monte['mission_propellant_fraction'], bins=24, color='tab:orange', edgecolor='white')
    ax3.axvspan(
        summary.loc[0, 'paper_propellant_fraction_min'],
        summary.loc[0, 'paper_propellant_fraction_max'],
        color='tab:orange',
        alpha=0.15,
        label='Paper concept PMF band',
    )
    ax3.set_title(f'{hybrid} Monte Carlo propellant fraction distribution')
    ax3.set_xlabel('Mission propellant mass fraction')
    ax3.set_ylabel('Count')
    ax3.legend()
    save_plot(fig3, output_dir / f'monte_carlo_distribution_{key}.png')
    plt.show()
    plt.close(fig3)

    fig4, ax4 = plt.subplots(figsize=(8, 5))
    ax4.hist(monte['transit_time_days'], bins=24, color='tab:purple', edgecolor='white')
    ax4.axvspan(
        summary.loc[0, 'paper_transit_min_days'],
        summary.loc[0, 'paper_transit_max_days'],
        color='tab:green',
        alpha=0.18,
        label='Paper concept transit band',
    )
    ax4.axvspan(
        summary.loc[0, 'baseline_reference_transit_min_days'],
        summary.loc[0, 'baseline_reference_transit_max_days'],
        color='tab:red',
        alpha=0.10,
        label='Baseline 180-270 days',
    )
    ax4.set_title(f'{hybrid} transit time histogram')
    ax4.set_xlabel('Transit time [days]')
    ax4.set_ylabel('Count')
    ax4.legend()
    save_plot(fig4, output_dir / f'transit_histogram_{key}.png')
    plt.show()
    plt.close(fig4)

    fig5, ax5 = plt.subplots(figsize=(9, 5))
    ax5.bar(profile['phase_name'], profile['thrust_multiplier'], color='tab:olive', alpha=0.75, label='Thrust multiplier')
    ax5.set_title(f'{hybrid} variable-mode phase profile')
    ax5.set_ylabel('Thrust multiplier')
    ax5.set_xlabel('Mission phase')
    ax5_twin = ax5.twinx()
    ax5_twin.plot(profile['phase_name'], profile['thermal_isp_s'], color='tab:gray', marker='o', linewidth=2, label='Thermal Isp')
    ax5_twin.plot(profile['phase_name'], profile['effective_isp_s'], color='tab:blue', marker='o', linewidth=2, label='Effective Isp')
    ax5_twin.set_ylabel('Isp [s]')
    lines1, labels1 = ax5.get_legend_handles_labels()
    lines2, labels2 = ax5_twin.get_legend_handles_labels()
    ax5.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
    save_plot(fig5, output_dir / f'thrust_profile_{key}.png')
    plt.show()
    plt.close(fig5)

    display(paper)
    display(summary)
    display(profile[['phase_name', 'mode_name', 'effective_isp_s', 'thrust_multiplier', 'phase_duration_days']])

def on_click(_):
    with output:
        output.clear_output()
        output_dir = run_simulation(hybrid_dropdown.value, dv_slider.value, 360)
        print(f'Saved CSVs and plots to: {output_dir}')
        frames = load_results(output_dir, hybrid_dropdown.value)
        render_plots(output_dir, hybrid_dropdown.value, frames)

run_button.on_click(on_click)
display(widgets.VBox([widgets.HBox([hybrid_dropdown, dv_slider, run_button]), output]))
